In [1]:
import sys
sys.path.append('c:/Pradap/Research/Python-Packages/enrique/')

In [2]:
import magellan as mg
import pandas as pd

from magellan.debug.decisiontree import visualize_tree, debug_dt
from magellan.feature.extractfeatures import apply_feat_fns

In [3]:
mg.init_jvm('C:\\Program Files\\Java\\jre7\\bin\\server\\jvm.dll')

A = mg.load_dataset('table_A')
B = mg.load_dataset('table_B')

In [4]:
ab = mg.AttrEquivalenceBlocker()
C = ab.block_tables(A, B, 'zipcode', 'zipcode', l_output_attrs=['name', 'hourly_wage', 'zipcode'],
                    r_output_attrs=['name', 'hourly_wage', 'zipcode'])


In [5]:
S = mg.sample_one_table(C, 10)
L = mg.load_table('../notebooks/demo_label.pkl')
L.set_property('ltable', A)
L.set_property('rtable', B)

In [6]:
feat_table = mg.get_features_for_blocking(A, B)
x = [4, 14, 17, 19, 10, 12]
feat_table = feat_table.ix[x]

In [7]:
S_prime = mg.extract_feat_vecs(L, attrs_after='label', feat_table=feat_table)


In [8]:
exclude_attrs=['_id', 'ltable.ID', 'rtable.ID', 'label']

In [9]:
rf = mg.RFMatcher(random_state=81)
rf.fit(table=S_prime, exclude_attrs=exclude_attrs, target_attr='label')

['ID_ID_exm', 'birth_year_birth_year_anm', 'hourly_wage_hourly_wage_anm', 'zipcode_zipcode_exm', 'name_name_nmw', 'name_name_swg']


In [10]:
clf = rf.clf

In [11]:
t1 = A.ix[2]
t2 = B.ix[1]

In [12]:
for e in clf.estimators_:
    debug_dt(e, t1, t2, feat_table, S_prime.columns, exclude_attrs, ensemble_flag=True)

    name_name_nmw <= 0.676339268684 is False (  value : 0.785714268684)
   Match status : True
   Prob. for non-match : 0.0
   Prob for match : 1.0
    name_name_nmw <= 0.65625 is False (  value : 0.785714268684)
   Match status : True
   Prob. for non-match : 0.0
   Prob for match : 1.0
    birth_year_birth_year_anm <= 0.999748468399 is False (  value : 1.0)
   Match status : True
   Prob. for non-match : 0.0
   Prob for match : 1.0
    birth_year_birth_year_anm <= 0.999748468399 is False (  value : 1.0)
   Match status : True
   Prob. for non-match : 0.0
   Prob for match : 1.0
    name_name_nmw <= 0.65625 is False (  value : 0.785714268684)
   Match status : True
   Prob. for non-match : 0.0
   Prob for match : 1.0
    hourly_wage_hourly_wage_anm <= 0.991666674614 is False (  value : 1.0)
   Match status : True
   Prob. for non-match : 0.0
   Prob for match : 1.0
    name_name_swg <= 0.51748251915 is False (  value : 0.781818211079)
   Match status : True
   Prob. for non-match : 0.

In [13]:
debug_dt(clf.estimators_[8], t1, t2, feat_table, S_prime.columns, exclude_attrs)

Match status : False


In [14]:
cols = [c not in exclude_attrs for c in S_prime.columns]
feature_names = S_prime.columns[cols]

In [19]:
feat_values = apply_feat_fns(t1, t2, feat_table)
feat_values = pd.Series(feat_values)
feat_values =  feat_values[feature_names]
v = feat_values.values
p = clf.predict_proba(v)
print feat_values
print p

ID_ID_exm                      0.000000
birth_year_birth_year_anm      1.000000
hourly_wage_hourly_wage_anm    1.000000
zipcode_zipcode_exm            1.000000
name_name_nmw                  0.785714
name_name_swg                  0.781818
dtype: float64
[[ 0.1  0.9]]


In [16]:
d = clf.estimators_[8]

In [17]:
d.tree_.children_right

array([-1], dtype=int64)